In [1]:
import pandas as pd
import re
import os
from collections import defaultdict

RuntimeError: module compiled against API version 0xa but this version of numpy is 0x9

#### JIRA BDD-1677  
**Goal:**  
Curation of clinical data SDY144

In [2]:
##DIRECTORY CELL

##checks to see if directory exists 
def exists(direc, file_name):
    exists_ornah = os.path.isdir(direc)
    while exists_ornah == False:
        direc = input("Directory/file does not exist.. please enter file directory, i.e '/home/name/Dektop/BDD2204/file.txt0 for the file -- " + file_name + ': \n')
        direc = os.path.normpath(direc)
        exists_ornah = os.path.isdir(direc)
    print('Current directory: ' + os.getcwd() + '\n')
    return(direc)

filepaths = []
examplefiles = ['mapping_subjectIDs.txt', 'clinical_data_BDD1677.txt', 'planned_visits.txt', 'assessment_prep.txt', 'adverse_events_prep.txt', 'interventions.txt']

beg = 'Please enter file path for '
mappfile = input(beg + 'mapping subjects file, such as ' + examplefiles[0] +': \n')
clinical_data = input(beg + "clinical data, such as  " + examplefiles[1] +': \n')
planned_visits = input(beg + "planned visits filepath, such as " + examplefiles[2] + ': \n')
template_assessment = input(beg + 'assessments, such as  '+ examplefiles[3] +': \n')
template_adverse_events = input(beg + 'adverse_events_prep , such as '  + examplefiles[4] +': \n')
template_interventions = input(beg + 'interventions file , such as '+ examplefiles[5] +': \n')

#NOTE: the input corrector is hard coded!!!
filepaths = [mappfile, clinical_data, planned_visits, template_assessment, template_adverse_events, template_interventions]

j = 0
for i in filepaths:
    exists(i, examplefiles[j])
    j += 1 

Please enter file path for mapping subjects file, such as mapping_subjectIDs.txt: 
/home/ale
Please enter file path for clinical data clinical_data_BDD1677.txt: 
/home/ale/asd
Please enter file path for planned visits filepathplanned_visits.txt: 
/home/ale
Please enter file path for assessments assessment_prep.txt: 
/home/ale
Please enter file path for adverse_events_prep adverse_events_prep.txt: 
/home/ale
Please enter file path for interventions file interventions.txt: 
/home/ale
Current directory: /home/ale/Documents/notebooks_round2

Directory does not exist.. please enter file directory, i.e '/home/name/Dektop/BDD2204/file.txt0 for the file -- clinical_data_BDD1677.txt: 
/home/ale
Current directory: /home/ale/Documents/notebooks_round2

Current directory: /home/ale/Documents/notebooks_round2

Current directory: /home/ale/Documents/notebooks_round2

Current directory: /home/ale/Documents/notebooks_round2

Current directory: /home/ale/Documents/notebooks_round2



In [3]:
## remap the subjects ID to subject ACC
#mappfile = "/home/cthomas/Desktop/cris/projects/BDD1677/mapping_subjectIDs.txt"
#clinical_data = "/home/cthomas/Desktop/cris/projects/BDD1677/clinical_data_BDD1677.txt"
#planned_visits = "/home/cthomas/Desktop/cris/projects/BDD1677/planned_visits.txt"

## get subject ID info
subjectsIDs = {}
study_acc = {}
ages = {}
with open(mappfile) as mpfl:
    for mpflln in mpfl:
        mpflln = mpflln.strip()
        mpfllnstf = mpflln.split("\t")
        subjectsIDs[mpfllnstf[1]] = mpfllnstf[0]
        study_acc[mpfllnstf[1]] = mpfllnstf[3]
        ages[mpfllnstf[1]] = mpfllnstf[2]

pv = defaultdict(dict)   ## pv[USER DEF VISIT ID][SDY] = PV ACC
pvdays = {}
with open(planned_visits, "r") as pvsts:
    for pvstsln in pvsts:
        pvstsln = pvstsln.strip()
        pvstslnstf = pvstsln.split("\t")
        pv[pvstslnstf[1]][pvstslnstf[2]] = pvstslnstf[0]
        pvdays[pvstslnstf[1]] = pvstslnstf[3]


IsADirectoryError: [Errno 21] Is a directory: '/home/ale'

In [ ]:
## prepare for adverse events, assessments and interventions:
#########
#template_assessment = "/home/cthomas/Desktop/cris/projects/BDD1677/files/assessment_prep.txt"
#template_adverse_events = "/home/cthomas/Desktop/cris/projects/BDD1677/files/adverse_events_prep.txt"

assessments = defaultdict(dict)
interventions = defaultdict(dict)
counter_ae = {"SDY522" : 0, "SDY144" : 0}
empty = ""

with open(clinical_data, "r") as clindata, open(template_adverse_events, "w") as ae:
    ae.write("UID\tsubID\tSDY\tname\t\tseverity\toutcome\tstartday\t\trelation_treatment\n")
    
    fl = clindata.readline()
    
    for line in clindata:
        line = line.strip("\n")
        data = line.split("\t")
        for i in range(len(data)):
            data[i] = data[i].strip()
        if data[12].endswith("ing"):
            data[12] = pvdays[data[1]]
        ID = "~~".join([data[0], data[1]])
        if data[11].endswith("es"):
            counter_ae[study_acc[data[0]]] += 1
            userAEid = "_".join(["HIPCBaylor_AE", study_acc[data[0]], 
                                 str(counter_ae[study_acc[data[0]]]).zfill(3)])
            ae.write("\t".join([userAEid, subjectsIDs[data[0]], study_acc[data[0]], data[13], empty,
                               "Not_Specified", "Not_Specified", data[12], empty, "Not_Specified"]))
            ae.write("\n")
        
        if data[2].strip():
            assessments[ID]["Height"] = data[2]
        if data[3].strip():
            assessments[ID]["Weight"] = data[3]
        if data[5].strip():
            assessments[ID]["Temperature"] = data[5]
        if data[4].strip():
            assessments[ID]["Temperature_Method"] = data[4]
        if data[6].strip():
            assessments[ID]["Pulse"] = data[6]
        if data[7].strip():
            assessments[ID]["Systolic"] = data[7]
        if data[8].strip():
            assessments[ID]["Diastolic"] = data[8]
        if data[9].strip():
            assessments[ID]["Respiratory_Rate"] = data[9]
        if data[14].strip():
            assessments[ID]["Had_FLU_Vaccine_Past"] = data[14]
        if data[15].strip():
            assessments[ID]["Vaccine_Type_Past"] = data[15]        
        if data[16].strip():
            assessments[ID]["Flu_Confirmed_Before"] = data[16]
        if data[17].strip():
            assessments[ID]["Current_Diagnosis"] = data[17] ## age unit at onset has to be a decimal. ...       
        if data[10].strip():
            interventions[ID]["RX_Past2Weeks"] = data[10]
        if data[18].strip():
            interventions[ID]["Current_Medications"] = data[18]



In [ ]:
## assessments:
counter_as = {"SDY522" : 0, "SDY144" : 0}
assmt_headers = "\tUID\tplanned_visitID\tcompname\tday\t\tage\tage_unit\t\t\t\tvalue\tunit\tcat\tpos\ttod\tq\twho"
flu_history = ["Had_FLU_Vaccine_Past", "Vaccine_Type_Past"] 
med_history = ["Flu_Confirmed_Before", "Current_Diagnosis"]
ast_vitals = ["Height", 
              "Weight", 
              "Temperature", 
              "Temperature_Method", 
              "Pulse", 
              "Systolic", 
              "Diastolic", 
              "Respiratory_Rate"]
na = "N/A"

with open(template_assessment, "w") as ast:
    ast.write("subID\tasstPanelID\tsdyID\tasstName\tasstType\tstd\tstatus\tcrf\tsepcol")
    ast.write("\t".join([assmt_headers * 24]))
    ast.write("\n")
    vitals = defaultdict(list)
   
    for allids in assessments:
        user_subID, pvID = allids.split("~~")
        hstry = []
        fluH = []
        for assmt in assessments[allids]:
            bidule = []
            assess = []
            if assessments[allids][assmt] == na:
                assess.append(assessments[allids][assmt])
            else: 
                assess = assessments[allids][assmt].split(r"/")
            for trucs in assess:
                counter_as[study_acc[user_subID]] += 1
                asscompID = "_".join(["HIPCBaylor", study_acc[user_subID], assmt, 
                                  str(counter_as[study_acc[user_subID]]).zfill(4)])
                bits = "\t".join([asscompID, pv[pvID][study_acc[user_subID]], assmt, pvdays[pvID], empty,
                                  ages[user_subID], "Years", empty, empty, empty, trucs,
                                  empty, empty, empty, empty, empty, empty])
                bidule.append(bits)

            newbit = "\t".join(bidule)        
            if assmt in ast_vitals:
                 vitals[user_subID].append(newbit)
            if assmt in flu_history:
                fluH.append(newbit)
            if assmt in med_history:
                hstry.append(newbit)

        if hstry:
            hsty = "\t".join(hstry)
            panelIDh = "_".join(["HIPCBaylor", study_acc[user_subID], "medical_history"])
            ast.write("\t".join([subjectsIDs[user_subID], panelIDh, study_acc[user_subID], "Medical History", 
                                 "Medical History", empty, empty, empty, empty, hsty]))
            ast.write("\n")
        if fluH:
            fluhist = "\t".join(fluH)
            panelIDf = "_".join(["HIPCBaylor", study_acc[user_subID], "vaccination_history"])
            ast.write("\t".join([subjectsIDs[user_subID], panelIDf, study_acc[user_subID], "Vaccination History", 
                                 "Medical History", empty, empty, empty, empty, fluhist]))
            ast.write("\n")

    for patients in vitals:
        if vitals[patients]:
            vital = "\t".join(vitals[patients])
            panelIDv = "_".join(["HIPCBaylor", study_acc[patients], "vital_signs"])
            ast.write("\t".join([subjectsIDs[patients], panelIDv, study_acc[patients], "Vital Signs", 
                                 "Other", empty, empty, empty, empty, vital]))
            ast.write("\n")


In [ ]:
## intervention:
    rx_past2Weeks
    current medication

In [ ]:
#template_intervention = "/home/cthomas/Desktop/cris/projects/BDD1677/files/interventions.txt"
counter_iv = {"SDY522" : 0, "SDY144" : 0}
with open(template_intervention, "w") as iv:
    ## prep template:
    iv.write("\t".join(["interventions","Schema Version 2.30","Content Version 2.30"]))
    iv.write("\n")
    iv.write("Please do not delete or edit this column\n")
    iv.write("\t".join(["Column Name","Intervention User-Defined ID","Subject ID","Study ID",
                        "Name","Compound Name","Compound Role","Dose Reported","Start Day",
                        "End Day","Status","Indication","Formulation","Dose","Dose Units",
                        "Dose Frequency per Interval","Route of Administration","Is Ongoing",
                        "Start Time","End Time","Start Day From Partial Date",
                        "End Day From Partial Date","Duration","Duration Unit"]))
    iv.write("\n")
    
    for idtites in interventions:
        user_ID, pvisitID = idtites.split("~~")
        for inter in interventions[idtites]:
            trc = interventions[idtites][inter].split(r"/")
        
            for meds in trc:
                counter_iv[study_acc[user_ID]] += 1
                start_day = pvdays[pvisitID]
                if inter.endswith("eeks"):
                    start_day = int(start_day) - 14
                ivID = "_".join(["HIPCBaylor_IV", study_acc[user_ID], str(counter_iv[study_acc[user_ID]]).zfill(3)])
                iv.write("\t".join([empty, ivID, subjectsIDs[user_ID], study_acc[user_ID], meds, meds,
                                   "Concomitant Medication", "UNK", str(start_day)]))
                iv.write("\t" * 6)
                iv.write("\n")